In [19]:
from bs4 import BeautifulSoup
import pandas as pd
import os
import time
import selenium
from selenium import webdriver
from selenium.common.exceptions import ElementClickInterceptedException
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
import numpy as np
from sklearn.linear_model import LinearRegression
import statsmodels.formula.api as smf
import math
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
import random
import csv
import seleniumwire
import re
import json
import pickle
from selenium.webdriver.common.keys import Keys

In [23]:
options = webdriver.ChromeOptions()
options.add_experimental_option("useAutomationExtension", False)
options.add_experimental_option("excludeSwitches",["enable-automation"])
driver_path = '/Users/dylan/Desktop/Y4S1/BT4103/scraper/chromedriver'
driver = webdriver.Chrome(executable_path=driver_path, chrome_options=options)
driver.delete_all_cookies()
driver.set_window_position(0,0)
driver.get('https://www.google.com/')
ID = 'Donald Trump'
driver.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(ID)
driver.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.RETURN)

<ipython-input-23-b7b4d3591f36>:5: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(executable_path=driver_path, chrome_options=options)


## 1) Import URL csv that contains the list of all URLs to scrape

In [ ]:
import csv

with open('url.csv', newline='') as f:
    reader = csv.reader(f)
    DATA = list(reader)

print(DATA)

## 2) Read in ID and password for quick login

In [ ]:
ID = "leowsy@comfortdelgrobus.com.sg"
password = "Cdgbmar21++"

## 3) Open chrome driver and navigate Tenderboard to deals page

In [ ]:
options = webdriver.ChromeOptions()
options.add_experimental_option("useAutomationExtension", False)
options.add_experimental_option("excludeSwitches",["enable-automation"])
driver_path = '/Users/dylan/comfort/Tender Prediction/chromedriver'
driver = webdriver.Chrome(executable_path=driver_path, chrome_options=options)
driver.delete_all_cookies()
driver.set_window_position(0,0)
driver.get('https://www.tenderboard.biz/login')
driver.find_element_by_id('edit-name').send_keys(ID)
driver.find_element_by_id('edit-pass').send_keys(password)
driver.find_element_by_xpath('//*[@id="user-login"]/div/div[4]/input').click()
time.sleep(5)
driver.find_element_by_xpath('//*[@id="menu-tenders"]').click()


## 4) Define helper helper functions for scraping

In [ ]:
def interceptor(request, ref):
    #def interceptor (request, referer_value)
    '''
    Changes the referer website to Tenderboard to prevent detection
    '''
    del request.headers['referer']  
    request.headers['referer'] = 'https://www.tenderboard.biz/tenders' 
    
def format_names(names,company_names):
    '''
    filter out company names only from the list of names in the div and appends
    to company_names to store all the company names
    '''
    for i in range(len(names)):
        word = names[i].text
        if word != '' and 'cipant' not in word:
            company_names.append(word)
    return company_names
            
def format_prices(nums,prices):
    '''
    filter out only the prices and stores into prices
    '''
    for i in range(len(nums)):
        word = nums[i].text
        if word != '' and len(word) <20:
            prices.append(word)
    return(prices)


def format_bids(prices, num_of_bid, bid_list):
    '''
    arranges the bids into lists for the individual respective companies
    '''
    temp_array = []
    for i in range (len(prices)):
        if len(temp_array) == num_of_bid:
            bid_list.append(temp_array)
            temp_array = []
            temp_array.append(prices[i])
        else:
            temp_array.append(prices[i])

        if len(temp_array) == num_of_bid:
            bid_list.append(temp_array)
            temp_array = []
    return bid_list

def add_error_to_log(url, error):
    '''
    logs the error that was met during scraping into a dictionary.
    the log will be of the format {'url1': [error1, error2,...]}
    '''
    global ERROR_LOG
    if url not in ERROR_LOG.keys():
        ERROR_LOG[url] = [error]
    else:
        ERROR_LOG[url].append(error)
        
        
def get_industry(lst):
    '''
    parses in selenium element and splits it into a list of elements.
    iteratively selects line items industry from the list.
    '''
    industry = ''
    line = []
    lst = lst[0].text.split('\n')
    for i in range(len(lst)):
        if lst[i].startswith('Industry:'):
            industry = lst[i+1]
    return industry 

def get_line(lst):
    '''
    parses in selenium element and splits it into a list of elements.
    iteratively selects line items from the list.
    '''
    industry = ''
    line = []
    lst = lst[0].text.split('\n')
    for i in range(len(lst)):
        if lst[i].startswith('Line Items:'):  
            for j in range(i+1,len(lst)):
                if lst[j].startswith('Item '):
                    line.extend(lst[j].split('\n'))
                elif lst[j].startswith('Files '):
                    break
    return line

def get_line_items():

    '''
    get_line_items gets the div from the website that contains all the line items
    the div then allocates the button header attribute if the button header is present
    if or otherwise, the scraper will carry out different scraping algorithms that will extract a list
    of all the line items in the div
    '''
    randwait = random.randrange(1,2) + random.choices([0.1,0.2,0.25])[0]
    randwait2 = random.randrange(1,2) + random.choices([0.1,0.2,0.25])[0]
    randwait3 = random.randrange(1,2) + random.choices([0.1,0.2,0.25])[0]
    more_details_class = driver.find_elements_by_xpath('//*[@id="tender-more-detail"]/div')
    button_header = None
    for i in more_details_class:
        class_value = i.get_attribute('class')
        if class_value == 'mdl-cell mdl-cell--10-col-desktop mdl-cell--hide-tablet mdl-cell--hide-phone':

            button_header = i

    selenium_list_items = []
    
    #if there is button (more than 4 line items):
    if button_header != None:
        for i in range(1,10):
            time.sleep(1)
            print(i)
            try:
                time.sleep(randwait)
                button_header2 = button_header.find_element_by_xpath('div')
                button_header3 = button_header2.find_element_by_xpath('button')
                time.sleep(randwait2)
                button_header3.click()
                time.sleep(randwait3)
                driver.find_element_by_xpath(f'//*[@id="tender-more-detail"]/div[6]/div/div/ul/li[{i}]').click()
                lst = driver.find_elements_by_xpath('//*[@id="tender-more-detail"]')
                selenium_list_items.append(get_line(lst))
                
            except NoSuchElementException as nse:
                break
    else:
        lst = driver.find_elements_by_xpath('//*[@id="tender-more-detail"]')
        selenium_list_items.append(get_line(lst))
    return flatmap(selenium_list_items)

def get_page_bids(bid_items):
    bid_items = bid_items[0]
            
    bid_prices_list = []
    
    bid_items = bid_items.text
    bid_items = bid_items.replace(',', '')
    bid_items = bid_items.replace('\n', 'mmm').split('mmm')
    
    for item in bid_items:
        try:
            flt = float(item)
            bid_prices_list.append(flt)
        except Exception as e:
            continue
            
    return bid_prices_list


def get_bid_items():
    randwait = random.randrange(1,2) + random.choices([0.1,0.2,0.25])[0]
    randwait2 = random.randrange(1,2) + random.choices([0.1,0.2,0.25])[0]
    randwait3 = random.randrange(1,2) + random.choices([0.1,0.2,0.25])[0]
    button_items = driver.find_elements_by_xpath('//*[@id="container"]/div/div[2]/div[2]/div[1]/div[6]/div')
    button_header = None
    
    for i in button_items:
        class_value = i.get_attribute('class')
        #print(class_value)
        if class_value == 'mdl-cell mdl-cell--4-offset-desktop mdl-cell--8-col-desktop mdl-cell--hide-tablet mdl-cell--hide-phone':
            #mdl-cell mdl-cell--2-offset-tablet mdl-cell--6-col-tablet mdl-cell--hide-desktop mdl-cell--hide-phone
            button_header = i
            
    bid_list = []
    
        #if there is button (more than 4 line items):
    if button_header != None:
        for i in range(1,10):
            time.sleep(1)
            print(i)
            try:
                time.sleep(randwait)
                button_header2 = button_header.find_element_by_xpath('div')
                button_header3 = button_header2.find_element_by_xpath('button')
                time.sleep(randwait2)
                button_header3.click()
                time.sleep(randwait3)
                driver.find_element_by_xpath(f'//*[@id="container"]/div/div[2]/div[2]/div[1]/div[6]/div[4]/div/div/ul/li[{i}]').click()
                #div that contains the names of the bidders, and the price
                bid_items = driver.find_elements_by_xpath('//*[@id="container"]/div/div[2]/div[2]/div[1]/div[6]')
                page_bids = get_page_bids(bid_items)
                bid_list.append(page_bids)
                
            except NoSuchElementException as nse:
                break
                
    else:
        bid_items = driver.find_elements_by_xpath('//*[@id="container"]/div/div[2]/div[2]/div[1]/div[6]')
        page_bids = get_page_bids(bid_items)
        bid_list.append(page_bids)
    
    return flatmap(bid_list)


def flatmap(lst):
    final = []
    for item in lst:
        for item2 in item:
            final.append(item2)
    return final


def organise(l, dim_r, dim_c,d):
    '''
    l, list of scraped numbers from site
    dim_r, number of rows (total number of companies)
    dim_c, number of columns (total number of bid items)
    d, dictionary of of {company: ['bid1', 'bid2'....]}
    
    skip calculates the indexes which we need to skip,
    which are the numbers that are not bids
    
    then, we append the rest of the bids to d, skipping
    all unwanted indexes.
    '''
    #indexes where we skip adding the item
    skip = int(len(l) / dim_r)
    current = 0
    processed_l = []
    for i in range(len(l)):
        if i % skip == 0:
            continue
        processed_l.append(l[i])

    current_row = 0
    while processed_l:
        for i in range(dim_c):
            d[current_row].append(processed_l.pop(0))
        current_row += 1


def resolve(l, num_rows, num_cols, d):
    '''
    recursive function that handles entire list,
    updating number of columns
    
    if:
    if the list l contains less than the number of rows*
    (4+1) (4 bids per page, plus 1 unwanted variable per 4 bids)
    
    else:
    if the length of l is more than num_rows*5,
    we process only the first (num_rows*(4+1)) entries in l,
    which are the number of entries for 1 page only.
    
    Then, we reduce the total number of columns by 4,
    as we have already processed 4 bid items.
    
    '''

    if len(l) <= num_rows*(4+1):
        organise(l, num_rows, num_cols, d)
    else:
        organise(l[:num_rows*(4+1)], num_rows, 4, d)
        resolve(l[num_rows*(4+1):], num_rows, num_cols-4, d)


## 5) Main scraping function that calls all previously defined functions 

In [ ]:
def scrape(url):
    '''
    scrape takes in the url (already navigated to by (3)). Scrapes all required values from each website by
    calling the functions defined earlier. Try and except clauses will allow all errors to be logged under 
    ERROR_LOG. Returns a list of required values in the following format:
    ['result', 'winners_bold', 'description', 'line_items', 'published_date', 'closing_date', 'closing_time', 
    'buyer_name', 'title','telephone', 'email','awarded_date', 'winner_text','awarded_amt','url']
    
    '''
    content = driver.page_source
    soup = BeautifulSoup(content, 'html.parser')
    names = soup.find_all(class_ ='mdl-cell mdl-cell--4-col mdl-cell--2-col-tablet mdl-cell--hide-phone')
    #nums = soup.find_all(class_  = 'mdl-cell mdl-cell--1-col mdl-cell--hide-desktop mdl-cell--hide-phone')

    winners_bold= ''
    title = ''
    items = ''
    published_date = ''
    closing_date = ''
    closing_time = ''
    indsutry = ''
    tel = ''
    email = ''
    buyer_name = ''
    line_items = ''
    company_names = []
    prices = []
    bid_list = []
    temp_array = []
    result = {}
    line = []
    
    try:
        line = get_line_items()
    except Exception as e:
        add_error_to_log(url,'line items')
    
    try:
        winners_bold = soup.find(style="font-weight: bold;")
    except Exception as e:
        add_error_to_log(url, 'winners_bold')

    try:
        title = driver.find_element_by_xpath('//*[@id="tender-desc"]/div[1]').text
    except Exception as e:
        add_error_to_log(url, 'title')
    
    try:
        items = soup.find_all(class_  = 'mdl-cell mdl-cell--6-col mdl-cell--hide-desktop mdl-cell--hide-phone')
    except Exception as e:
        add_error_to_log(url, 'items')
        
    try:    
        published_date = driver.find_element_by_xpath('//*[@id="tender-overview"]/div[6]').text
    except Exception as e:
        add_error_to_log(url, 'published date')

    try:    
        closing_date = driver.find_element_by_xpath('//*[@id="tender-overview"]/div[9]').text
    except Exception as e:
        add_error_to_log(url, 'closing date')
        
    try:    
        closing_time = driver.find_element_by_xpath('//*[@id="tender-overview"]/div[12]').text
    except Exception as e:
        add_error_to_log(url, 'closing time')

    try:     
        industry = driver.find_element_by_xpath('//*[@id="tender-more-detail"]/div[3]')
    except Exception as e:
        add_error_to_log(url, 'industry')

    try:    
        tel = driver.find_element_by_xpath('//*[@id="container"]/div/div[2]/div[2]/div[1]/div[4]/div[6]')
    except Exception as e:
        add_error_to_log(url, 'tel')
    
    try:
        email = driver.find_element_by_xpath('//*[@id="container"]/div/div[2]/div[2]/div[1]/div[4]/div[9]')
    except Exception as e:
        add_error_to_log(url, 'email')

    try:
        header = driver.find_element_by_xpath('//*[@id="tender-desc"]/div[1]').text
    except Exception as e:
        add_error_to_log(url, 'header')

    try:   
        title = driver.find_element_by_xpath('//*[@id="container"]/div/div[2]/div[2]/div[1]/div[4]/div[3]').text
    except Exception as e:
        add_error_to_log(url, 'title')

    try:
        awarded_date = driver.find_element_by_xpath('//*[@id="container"]/div/div[2]/div[2]/div[1]/div[7]/div[3]').text
        winner_text = driver.find_element_by_xpath('//*[@id="container"]/div/div[2]/div[2]/div[1]/div[7]/div[6]').text
        awarded_amt = driver.find_element_by_xpath('//*[@id="container"]/div/div[2]/div[2]/div[1]/div[7]/div[9]').text
    except Exception as e:
        add_error_to_log(url, 'winner data')
        
    #number of line items 
    num_cols = len(line)
    
    prices = get_bid_items()
   
    company_names = format_names(names,company_names)
    
    num_rows = len(company_names)
    
    num_of_bid = len(prices)/len(company_names)
    
    bid_list = format_bids(prices, num_of_bid, bid_list)

    buyer_name = driver.find_element_by_xpath('//*[@id="top-button-wrapper"]/div[1]').text
    
    lst = driver.find_elements_by_xpath('//*[@id="tender-more-detail"]')
    
    industry = get_industry(lst)
    
    item_values_by_company = {i:[] for i in range(num_rows)}
    resolve(prices, num_rows, num_cols,item_values_by_company)
    item_values_by_company = {company_names[k]:v for k,v in item_values_by_company.items()}

    
    final = [item_values_by_company, winners_bold.get_text(), industry, line, published_date, closing_date,closing_time, buyer_name, title,tel.text,email.text,awarded_date, winner_text,awarded_amt]
    
    return final


## 6) Create data structure for the storage of scraped data
## SCRAPED_DATA: list of lists that contains the data scraped from each website
## SCRAPED_URLS: contains all urls that have been scraped for logging purposes
## ERROR_LOG: logs urls with errors and the error faced
## ROUND_COUNTER: the number of rounds of scraping done
## please manually input round counter

In [ ]:
SCRAPED_DATA = []
SCRAPED_URLS = []
ERROR_LOG = {}
ROUND_COUNTER = 4
#DATAFRAME = pd.DataFrame


## 7) Scrape URLs by manually entering index. Scraped indexes will be logged seperately.
## Scraped data will be added to SCRAPED_DATA
## Scraped data does not automatically refresh. If you want to refresh,
## please run previous cell again

In [ ]:
def scrape_urls(start_index, end_index):
    '''
    scrapes 
    '''
    global ROUND_COUNTER
    global SCRAPED_DATA
    global ERROR_LOG
    global SCRAPED_URLS
    #global DATAFRAME
    for i in range(start_index, end_index):
        
        time.sleep(2)
        randwait = random.randrange(10,25) + random.choices([0.1,0.2,0.25])[0] + random.choices([0.37,0.48,0.63])[0]
        randwait2 = random.randrange(9,14) + random.choices([0.1,0.2,0.25])[0] + random.choices([0.37,0.48,0.63])[0]
        randwait3 = random.randrange(3,5) + random.choices([0.1,0.2,0.25])[0] + random.choices([0.37,0.48,0.63])[0]
        time.sleep(randwait2)

        url = DATA[i][0]
        print(url)
        driver.request_interceptor = interceptor
        time.sleep(randwait3)
        driver.get(url)

        driver.switch_to.window(driver.window_handles[0])
        time.sleep(randwait3)
        
        try:
            page_result = scrape(url)
        except Exception as e:
            print(url + "has an error")
        page_result.append(url)
        SCRAPED_DATA.append(page_result)
        SCRAPED_URLS.append(url)

        time.sleep(randwait)
        print(SCRAPED_DATA)
        print(i)
        
    print('done scraping')
    fields = ['result', 'winners_bold', 'description', 'line_items', 'published_date', 'closing_date', 'closing_time', 'title','buyer_name', 'telephone', 'email','awarded_date', 'winner_text','awarded_amt','url','drop']
    
    pickle_name = f'scrape_{ROUND_COUNTER}'
    
    
    #SCRAPED_DATA = []
    
    
    df = pd.DataFrame(SCRAPED_DATA, columns = fields)
    df = df[df.columns[:15]]
    df.to_pickle(pickle_name)
    
    log_data = SCRAPED_URLS.append(ERROR_LOG)
    
    #csv_name = f'scrape_18mar{ROUND_COUNTER}.csv'
    with open(f'url_error_log{ROUND_COUNTER}.csv', 'w') as f:
        write = csv.writer(f)
        for item in SCRAPED_URLS:
            write.writerow([item])
    ROUND_COUNTER += 1
    SCRAPED_URLS = []
    ERROR_LOG = {}
        
    

## 8. Carry out scraping
## Please manually input scraping URLS
## This will start the scraping process and potentially get TB Account Banned.
## Please DO NOT ANYHOW RUN

In [ ]:
print(ROUND_COUNTER)
scrape_urls(100,110)

## After scraping please convert scraped data into .pkl file
## Remember to run this code after each scraping round
## to prevent data loss

In [ ]:
df = pd.DataFrame(SCRAPED_DATA, columns = fields)
df.to_pickle('scrape_3')
    

## Manual function to scrape driver windows

In [ ]:
driver.switch_to.window(driver.window_handles[0])

In [ ]:
driver.switch_to.window(driver.window_handles[1])

## Functions to help you to combine pkl files and read as pd DataFrame

In [ ]:
#df = pd.DataFrame(SCRAPED_DATA, columns = fields)
df.to_pickle('scrape_4')
    

In [ ]:
with open('scrape_4', 'rb') as f:
    x = pickle.load(f)
    
with open('scrape_3', 'rb') as f:
    y = pickle.load(f)

with open('scrape_2', 'rb') as f:
    z = pickle.load(f)
    
with open('scrape_1', 'rb') as f:
    w = pickle.load(f)

In [ ]:
y.columns = ['result', 'winners_bold', 'description', 'line_items', 'published_date', 'closing_date', 'closing_time', 'title','buyer_name', 'telephone', 'email','awarded_date', 'winner_text','awarded_amt','url']#,'drop','drop']
z.columns = ['result', 'winners_bold', 'description', 'line_items', 'published_date', 'closing_date', 'closing_time', 'title','buyer_name', 'telephone', 'email','awarded_date', 'winner_text','awarded_amt','url']#,'drop','drop']
w.columns = ['result', 'winners_bold', 'description', 'line_items', 'published_date', 'closing_date', 'closing_time', 'title','buyer_name', 'telephone', 'email','awarded_date', 'winner_text','awarded_amt','url']#,'drop','drop']

In [ ]:
df = pd.concat([x,y], axis=0)
df = pd.concat([df,z], axis = 0)
df = pd.concat([df,w], axis = 0)
df

In [ ]:
x.to_pickle('tenderboard_dataset')

In [ ]:
x.iloc[2]["line_items"]

In [ ]:
x.iloc[5]["result"]

In [ ]:
x.iloc[1]["winner_text"]

In [ ]:
df['winner_text'].value_counts()
